In [36]:
import numpy as np
from cosmoTransitions import generic_potential_1
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
import scipy.integrate as integrate
from scipy import interpolate


####Some definitions##
v2 = 246.**2 
mh=125
v=246
alpha=1/137 #fine structure constant
sinthw=np.sqrt(0.223) #sine of Weinberg angle
g1=np.sqrt(4*np.pi*alpha/(1-sinthw**2)) ## U(1)_Y gauge coupling constant
g=np.sqrt(4*np.pi*alpha)/sinthw #SU(2)_L gauge coupling constant
Mplanck=2.4*10**18 #reduced planck mass
cs=1/3**0.5 ##Sound speed constant
me_phys=0.51**(-3)




class model1(generic_potential_1.generic_potential):
    """
    A sample model which makes use of the *generic_potential* class.

    This model doesn't have any physical significance. Instead, it is chosen
    to highlight some of the features of the *generic_potential* class.
    It consists of two scalar fields labeled *phi1* and *phi2*, plus a mixing
    term and an extra boson whose mass depends on both fields.
    It has low-temperature, mid-temperature, and high-temperature phases, all
    of which are found from the *getPhases()* function.
    """
    def init(self, flavonvevs, flavorscale):
        """
          m1 - tree-level mass of first singlet when mu = 0.
          m2 - tree-level mass of second singlet when mu = 0.
          mu - mass coefficient for the mixing term.
          Y1 - Coupling of the extra boson to the two scalars individually
          Y2 - Coupling to the two scalars together: m^2 = Y2*s1*s2
          n - degrees of freedom of the boson that is coupling.
        """
        # The init method is called by the generic_potential class, after it
        # already does some of its own initialization in the default __init__()
        # method. This is necessary for all subclasses to implement.

        # This first line is absolutely essential in all subclasses.
        # It specifies the number of field-dimensions in the theory.
        self.Ndim = 1

        # self.renormScaleSq is the renormalization scale used in the
        # Coleman-Weinberg potential.
        self.renormScaleSq = 173 #Top quark mass

        # This next block sets all of the parameters that go into the potential
        # and the masses. This will obviously need to be changed for different
        # models.
        self.yt=1
        self.ye=1
        self.vs=flavonvevs
        self.Mf=flavorscale
        self.ne=np.log(2**0.5*me_phys/self.ye/v)/np.log(self.vs/self.Mf)

 
       

    def forbidPhaseCrit(self, X):
        """
        forbidPhaseCrit is useful to set if there is, for example, a Z2 symmetry
        in the theory and you don't want to double-count all of the phases. In
        this case, we're throwing away all phases whose zeroth (since python
        starts arrays at 0) field component of the vev goes below -5. Note that
        we don't want to set this to just going below zero, since we are
        interested in phases with vevs exactly at 0, and floating point numbers
        will never be accurate enough to ensure that these aren't slightly
        negative.
        """
        return any(np.array([X])[...,0] < -5.0)
    


    def V0(self, X):
        """
        This method defines the tree-level potential. It should generally be
        subclassed. (You could also subclass Vtot() directly, and put in all of
        quantum corrections yourself).
        """
        # X is the input field array. It is helpful to ensure that it is a
        # numpy array before splitting it into its components.
        X = np.asanyarray(X)
        # x and y are the two fields that make up the input. The array should
        # always be defined such that the very last axis contains the different
        # fields, hence the ellipses.
        # (For example, X can be an array of N two dimensional points and have
        # shape (N,2), but it should NOT be a series of two arrays of length N
        # and have shape (2,N).)
        h = X[...,0]
        pot =-1/4*mh**2*h**2 + mh**2*h**4/(8*v**2)
        


        return pot

    
    def boson_massSq(self, X, T):
        X = np.array(X)
        h = X[...,0]

        #####Scalar thermal masses, obtained from eq. of 1809.08242 A.5
        Pi_h = T**2*(mh**2  +  2*g**2*v**2/4+  v**2/4*(g**2+g1**2) + 2*h**2/2)/4/v**2 
     
        ##Scalar mass matrix eq. (5) of 0711.2511##
        mh_phi=-mh**2/2 + 3*mh**2*h**2/v**2
        mh_phi+=Pi_h
        
 
        ####Gauge boson masses
        mW_phi = g**2*h**2/4
        mZ_phi =(g**2+g1**2)*h**2/4
   
        ####Gauge boson masses
        mW = g**2*h**2/4 + 11/6*g**2*T**2
        ag=g**2*h**2/4 + 11/6*g**2*T**2
        bg=1/4*g1**2*h**2 + 11/6*g1**2*T**2
        ccg=-1/4*g1*g*h**2
        Ag=(ag+bg)/2
        Bg=1/2*np.sqrt((ag-bg)**2+4*ccg**2)
        mZ=Ag+Bg
        mPh=Ag-Bg
      
        M = np.array([mh_phi,mW,mZ])
        Mphys = np.array([mh**2,g**2*v**2/4,v**2/4*(g**2+g1**2)])

        # At this point, we have an array of boson masses, but each entry might
        # be an array itself. This happens if the input X is an array of points.
        # The generic_potential class requires that the output of this function
        # have the different masses lie along the last axis, just like the
        # different fields lie along the last axis of X, so we need to reorder
        # the axes. The next line does this, and should probably be included in
        # all subclasses.
        M = np.rollaxis(M, 0, len(M.shape))
        Mphys = np.rollaxis(Mphys, 0, len(Mphys.shape))

        # The number of degrees of freedom for the masses. This should be a
        # one-dimensional array with the same number of entries as there are
        # masses.

        dof = np.array([1,6,3])

        # c is a constant for each particle used in the Coleman-Weinberg
        # potential using MS-bar renormalization. It equals 1.5 for all scalars
        # and the longitudinal polarizations of the gauge bosons, and 0.5 for
        # transverse gauge bosons.
        #c = np.array([1.5,1.5,1.5,1.5,1.5,1.5,1.5])
        c = np.array([1.5,1.5,1.5])
   
        return M, dof, c, Mphys


    
    def fermion_massSq(self, X):
        X = np.array(X)
        h = X[...,0]

        """
        Calculate the fermion particle spectrum. Should be overridden by
        subclasses.

        Parameters
        ----------
        X : array_like
            Field value(s).
            Either a single point (with length `Ndim`), or an array of points.

        Returns
        -------
        massSq : array_like
            A list of the fermion particle masses at each input point `X`. The
            shape should be such that  ``massSq.shape == (X[...,0]).shape``.
            That is, the particle index is the *last* index in the output array
            if the input array(s) are multidimensional.
        degrees_of_freedom : float or array_like
            The number of degrees of freedom for each particle. If an array
            (i.e., different particles have different d.o.f.), it should have
            length `Ndim`.

        Notes
        -----
        Unlike :func:`boson_massSq`, no constant `c` is needed since it is
        assumed to be `c = 3/2` for all fermions. Also, no thermal mass
        corrections are needed.
        """
        
        mt=self.yt**2*h**2/2
        me=self.ye**2*h**2/2*(self.vs*(np.cos(h*np.pi/2/v)+1))**self.ne
        
        M = np.array([mt,me])
        Mphys = np.array([v**2/2,me])

        # At this point, we have an array of boson masses, but each entry might
        # be an array itself. This happens if the input X is an array of points.
        # The generic_potential class requires that the output of this function
        # have the different masses lie along the last axis, just like the
        # different fields lie along the last axis of X, so we need to reorder
        # the axes. The next line does this, and should probably be included in
        # all subclasses.
        M = np.rollaxis(M, 0, len(M.shape))
        Mphys = np.rollaxis(Mphys, 0, len(Mphys.shape))

        # The number of degrees of freedom for the masses. This should be a
        # one-dimensional array with the same number of entries as there are
        # masses.
        dof = np.array([12])
        return M, dof, Mphys
 
 
    def approxZeroTMin(self):
        #There are generically two minima at zero temperature in this model,
        #and we want to include both of them.
        return [np.array([v])]
    
    


In [37]:
#ne=np.log(2**0.5*me/1/v)/np.log(0.2)
m=model1(200,300)

In [38]:
m.Vtot([0.2],0)

-7.3293243545994635e+34

In [24]:
m.getPhases()

Tracing phase starting at x = [152.66603779] ; t = 0.0
Tracing minimum up
traceMinimum t0 = 0


AttributeError: 'float' object has no attribute 'log'

In [166]:
#nQ=(3,2,0)
#nu=(3,2,0)
#nd=(1,0,0)

#nQ=(3,2,0)
#nu=(3,2,0)
#nd=(2,1,1)

nQ=(2,1,0)
nu=(5,2,0)
nd=(5,4,2)

mu=2.3*10**(-3)
deltamu=0.6*10**(-3)
mc=1.275
deltamc=0.025
mt=160
deltamt=4.5
md=4.8*10**(-3)
deltamd=0.4*10**(-3)
ms=95*10**(-3)
deltams=5*10**(-3)
mb=4.18
deltamb=0.03
me=5.11*10**(-4)
deltame=5.11*10**(-6)
mmu=0.106
deltammu=0.106*10**(-2)
mtau=1.78
deltamtau=1.78*10**(-2)
Vus=0.225
deltaVus=0.225*10**(-2)
Vub=3.55*10**(-3)
deltaVub=0.15*10**(-3)
Vcb=4.14*10**(-2)
deltaVcb=1.2*10**(-3)


def Yukawa_mat(yij,nQ,nq,epsilon):
    """Generic Yukawa matrix ( times v/2**0.5 ) given numerical coefficients, U(1) charges and 
    ratio of flavon vev to flavor scale"""
    columns=[]
    for i in range(3):
        rows=[]
        for j in range(3):
            rows.append(yij[i,j]*epsilon**(nQ[i]+nq[j]))
        columns.append(rows)
    return np.array(columns)*v/2**0.5


def massesup(yij,nQ,nq,epsilon):
    """Returns the masses given numerical coefficients, U(1) charges and 
    ratio of flavon vev to flavor scale"""
    YQ=Yukawa_mat(yij,nQ,nq,epsilon)   
    YQ_dagger=YQ.T
    YY_dagger=np.matmul(YQ,YQ_dagger)
    w, v= np.linalg.eigh(YY_dagger)
    mass=[]
    for i in w:
        mass.append(np.sqrt(i))
    return mass


def chi_square_log(uparams,nQ,nq,epsilon):
    """Optimization function to find best fit value of the up sector"""
    u1,u2,u3,u4,u5,u6,u7,u8,u9=uparams
    yij=np.array([[u1,u2,u3],[u4,u5,u6],[u7,u8,u9]])
    return (massesup(yij,nQ,nq,epsilon)[0]-mu)**2/deltamu**2 \
           +(massesup(yij,nQ,nq,epsilon)[1]-mc)**2/deltamc**2 \
           +(massesup(yij,nQ,nq,epsilon)[2]-mt)**2/deltamt**2 \
           +(np.log(abs(u1)))**2/np.log(3)**2 \
           +(np.log(abs(u2)))**2/np.log(3)**2 +(np.log(abs(u3)))**2/np.log(3)**2 \
           +(np.log(abs(u4)))**2/np.log(3)**2 +(np.log(abs(u5)))**2/np.log(3)**2 \
           +(np.log(abs(u6)))**2/np.log(3)**2 +(np.log(abs(u7)))**2/np.log(3)**2 \
           +(np.log(abs(u8)))**2/np.log(3)**2 +(np.log(abs(u9)))**2/np.log(3)**2 




In [210]:
for i in range(100):
    uparams=np.ones(9)+np.random.uniform(-.3,.3,9)
    sol_fit=optimize.minimize(chi_square_log,x0=uparams,args=(nQ,nu,0.2))
    if sol_fit.fun<=12:
        break

In [211]:
u1,u2,u3,u4,u5,u6,u7,u8,u9=sol_fit.x
yij=np.array([[u1,u2,u3],[u4,u5,u6],[u7,u8,u9]])
massesup(yij,nQ,nu,.2)

[0.0018351395249486368, 1.2882410419249317, 163.3291191590386]

In [ ]:
massesup(yij,nQ,nq,epsilon)